In [273]:
import pandas as pd

df = pd.read_csv('../train.csv')
df.size

1910006

In [274]:
df.isna().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [275]:
df.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [276]:
df['Type'].unique()

array(['L', 'M', 'H'], dtype=object)

In [277]:

count_duplicates = df.duplicated().sum()


print("Number of duplicate records:", count_duplicates)

df.drop_duplicates(inplace=True)

Number of duplicate records: 0


In [278]:
df.columns

Index(['id', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [279]:
df.drop(['id', 'Product ID'], axis=1, inplace=True)

In [280]:
df  = pd.get_dummies(df)

In [281]:
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,300.6,309.6,1596,36.1,140,0,0,0,0,0,0,0,1,0
1,302.6,312.1,1759,29.1,200,0,0,0,0,0,0,0,0,1
2,299.3,308.5,1805,26.5,25,0,0,0,0,0,0,0,1,0
3,301.0,310.9,1524,44.3,197,0,0,0,0,0,0,0,1,0
4,298.0,309.0,1641,35.4,34,0,0,0,0,0,0,0,0,1


In [282]:
df.columns

Index(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]',
       'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Type_H',
       'Type_L', 'Type_M'],
      dtype='object')

In [283]:
cols = ['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF', 'Type_H',
       'Type_L', 'Type_M','Machine failure']
df = df[cols]

df['Air temperature'] = df['Air temperature [K]']
df['Process temperature'] = df['Process temperature [K]']
df['Rotational speed'] = df['Rotational speed [rpm]']
df['Torque'] = df['Torque [Nm]']
df['Tool wear']=df['Tool wear [min]']

df.drop(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'], axis=1,inplace=True)

df.dropna(inplace=True)

In [284]:
X = df.drop('Machine failure', axis=1)
y=df['Machine failure']

In [285]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import log_loss

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.1, random_state=42)

In [286]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    'objective': 'binary:logistic',
    'seed': 1994
}

model = xgb.train(params,dtrain, num_boost_round=100,)



In [287]:
y_val_pred = model.predict(dval)
val_log_loss = log_loss(y_val, y_val_pred)
print('Validation Log Loss:', val_log_loss)

Validation Log Loss: 0.020364225508613148


In [288]:
dftest = pd.read_csv('../test.csv')
dftest

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,0,0,0,0,0
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,0,0,0,0,0
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,0,0,0,0,0
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,0,0,0,0,0


In [289]:
dftest.drop(['id', 'Product ID'], axis=1, inplace=True)
dftest  = pd.get_dummies(dftest)
dftest.head()


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,302.3,311.5,1499,38.0,60,0,0,0,0,0,0,1,0
1,301.7,311.0,1713,28.8,17,0,0,0,0,0,0,1,0
2,301.3,310.4,1525,37.7,96,0,0,0,0,0,0,1,0
3,300.1,309.6,1479,47.6,5,0,0,0,0,0,0,0,1
4,303.4,312.3,1515,41.3,114,0,0,0,0,0,0,0,1


In [290]:
dftest['Air temperature'] = dftest['Air temperature [K]']
dftest['Process temperature'] = dftest['Process temperature [K]']
dftest['Rotational speed'] = dftest['Rotational speed [rpm]']
dftest['Torque'] = dftest['Torque [Nm]']
dftest['Tool wear']=dftest['Tool wear [min]']

dftest.drop(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'], axis=1,inplace=True)

In [291]:
dtest = xgb.DMatrix(dftest)

test_pred = model.predict(dtest)

In [292]:
# df2 = pd.DataFrame(a, columns =['class_0', 'class_1']) 
test = pd.read_csv('../test.csv')
submission_df = pd.DataFrame({
    'id': test['id'],
    'Machine Failure': test_pred
})


In [293]:
submission_df.head()

,id,Machine Failure
0,136429,0.000565
1,136430,0.002690
2,136431,0.000149
3,136432,0.000125
4,136433,0.000219


In [294]:
submission_df.to_csv('submission.csv', index=False)